<a href="https://colab.research.google.com/github/Pathway2008/law_predict/blob/main/law_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/law/train.csv')
test = pd.read_csv('/content/drive/MyDrive/law/test.csv')
sub = pd.read_csv('/content/drive/MyDrive/law/sample_submission.csv')

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.9 MB/s eta 0:00:00


In [5]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import BCEWithLogitsLoss
from transformers import AdamW

In [6]:
# Load and preprocess the data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:
train_texts = train['facts'].tolist()
train_labels = train['first_party_winner'].tolist()

In [8]:
# Tokenize and encode the text data
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in train_texts]

input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)
labels = torch.tensor(train_labels)

In [9]:
# Split the data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_masks, labels, test_size=0.2, random_state=42)

In [10]:
# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [11]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [12]:
# Set the device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [13]:
# Set up the optimizer and loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = BCEWithLogitsLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [14]:
from tqdm import tqdm

epochs = 5

for epoch in range(epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        optimizer.zero_grad()

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': loss.item()})

    avg_loss = total_loss / len(train_dataloader)
    print(f'Epoch {epoch + 1}/{epochs} - Loss: {avg_loss:.4f}')

Epoch 1/5 - Loss: 0.6446


Epoch 2/5 - Loss: 0.6235


Epoch 3/5 - Loss: 0.5455


Epoch 4/5 - Loss: 0.3719


Epoch 5/5 - Loss: 0.1800


In [15]:
# Evaluation on the validation set
model.eval()
val_loss = 0
val_accuracy = 0

with torch.no_grad():
    for batch in val_dataloader:
        batch_inputs = batch[0].to(device)
        batch_masks = batch[1].to(device)
        batch_labels = batch[2].to(device)

        outputs = model(batch_inputs, attention_mask=batch_masks, labels=batch_labels)
        loss = outputs.loss
        logits = outputs.logits

        val_loss += loss.item()

        predictions = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(predictions == batch_labels).item()

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = val_accuracy / len(val_dataset) * 100

    print(f'Validation Loss: {avg_val_loss:.4f}')
    print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Loss: 1.2965
Validation Accuracy: 52.22%


In [17]:
test_texts = test['facts'].tolist()

In [18]:
tokenized_texts = [tokenizer.encode_plus(text, add_special_tokens=True, truncation=True, max_length=512,
                                         padding='max_length', return_tensors='pt') for text in test_texts]

In [19]:
input_ids = torch.cat([text['input_ids'] for text in tokenized_texts], dim=0)
attention_masks = torch.cat([text['attention_mask'] for text in tokenized_texts], dim=0)

In [21]:
test_dataset = TensorDataset(input_ids,attention_masks)

In [22]:
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True)

In [25]:
import numpy as np

In [26]:
# Set the model to evaluation mode
model.eval()

# List to store the predictions
predictions = []

# Iterate over the test dataloader
for batch in test_dataloader:
    batch_inputs = batch[0].to(device)
    batch_masks = batch[1].to(device)

    # Disable gradient calculation
    with torch.no_grad():
        # Forward pass
        outputs = model(batch_inputs, attention_mask=batch_masks)
        logits = outputs.logits

        # Apply softmax to convert logits to probabilities
        probabilities = torch.softmax(logits, dim=1)

        # Get the predicted labels
        _, predicted_labels = torch.max(probabilities, dim=1)

        # Append the predicted labels to the predictions list
        predictions.extend(predicted_labels.cpu().tolist())

# Convert predictions to a numpy array if needed
predictions = np.array(predictions)

# Print the predicted labels
print(predictions)

[1 1 0 ... 1 0 0]


In [28]:
sub['first_party_winner'] = predictions
sub['first_party_winner'].value_counts()

1    790
0    450
Name: first_party_winner, dtype: int64

In [29]:
sub.to_csv('sub_bert.csv', index = False)
#accuracy = 0.53548